## Introduction

Given the simplified table schemas provided below, write executable SQL queries to answer each question below:

1. List the top 20 cities by percentage share of total bookings between 2017-01-01 and 2017-08-30 (be sure to include each city’s share in your output)
2. For each of these top 20 cities, list their respective percentages of bookings that were cancelled between 2017-01-01 and 2017-08-30
3. For each week between 2017-01-01 and 2017-08-30, list the percentage breakdown of bookings made by repeat customers vs. first-time customers w/ promotion vs. first-time customers w/o promotion

**Table name: reservation**

| Column Name        | Data Type           |
| ------------- |:-------------:|
| id      | INT |
| search_request_id      | INT (foreign keyed to search_request.id) |
| date_created | TIMESTAMP |
| date_cancelled | TIMESTAMP (assume NULL if not cancelled)|
| user_id | INT |

**Table name: search_request**



| Column Name        | Data Type           |
| ------------- |:-------------:|
| id      | INT |
| date_created | TIMESTAMP |
| city_id | INT |
| promotion_id | INT (assume NULL if promotion is not applicable)|
| rooms | INT |
| check_in | TIMESTAMP |
| check_out | TIMESTAMP |

Note: each reservation originates from a search request, and may be joined on `reservation.search_request_id = search_request.id`

- We will use two additional packages for this case study, one is called [faker](https://github.com/joke2k/faker), which will be used to generate fake data for the challenge. The other one is called [pandasql](https://github.com/yhat/pandasql) that enables us to run sql query on a pandas dataframe by creating the temporary database using sqlite.
- pip install these two packages and restart the kernel. 

## Prepare the data

- We will start with the search request table first since the reservation table could be just a subset of it.
 - For integers, use `np.random.randint`
 - For random sampling, use `np.random.choice`
 - For timestamp, use [date_between_dates](http://faker.readthedocs.io/en/master/providers/faker.providers.date_time.html?highlight=date_between_dates) method of the Faker object
 - Not all the search has a promotion id so you can use `np.random.binomial` to randomly pick the ones w/ promotion and leave the others as `np.nan`. When you run SQL queries, it will be treated the same as `NULL` in SQL.
 - Use the `promotion_rate` as the probability for the binomial distribution. 

In [12]:
import datetime as dt
from datetime import timedelta
import numpy as np
import pandas as pd
from faker import Faker

In [17]:
# Fix random seed to make the result reproducible
np.random.seed(0)

num_users = 2000 # Number of users
num_search = 5000 # Number of searches
num_reservation = int(num_search * 0.1) # Assume 10% of the search got converted to reservation
num_cities = 120  # Number of cities
num_promotion = 20 # Number of different promotions
num_rooms = 5  # Largest reservation
most_days_in_advanced = 50 # customer can book at most 50 days after the search date to check in
most_days_stay = 30 # max stay to 8 days
promotion_rate = 0.1 # 10% of the people use promotion

# search_id
search_id = np.random.choice(range(1, int(1e6)), size = num_search, replace=False)

# city_id
city_id = np.random.randint(1, num_cities, size = num_search)

# number of rooms
rooms = np.random.randint(1,num_rooms, size = num_search)

# search_date
# Start date and end date of the fake data
start_date = dt.date(2016,6,6)
end_date = dt.date(2017,10,10)
# Use the `date_between_dates` method of a Faker object
f = Faker()
# Your code here
search_date = []
for _ in range(num_search):
    search_date.append(f.date_between_dates(start_date, end_date))

# promotion_id
# use `np.random.binomial` to randomly pick the ones w/ promotion
# Your code here
promotion_id = []
for _ in range(num_search):
    promotion_id.append(np.random.binomial(1,promotion_rate))

# check_in & check_out date
# use `dt.timedelta` function to create a time difference and add it to the original date
# Your code here
check_in = []
check_out = []
for searchdate in range(len(search_date)):
    time_to_checkin = search_date[searchdate] + timedelta(days = np.random.randint(1, most_days_in_advanced))
    time_to_checkout = time_to_checkin + timedelta(days = np.random.randint(1, most_days_stay))
    check_in.append(time_to_checkin)
    check_out.append(time_to_checkout)
print(check_in[:10])
print(check_out[:10])

[datetime.date(2016, 7, 8), datetime.date(2016, 10, 6), datetime.date(2016, 9, 5), datetime.date(2017, 10, 17), datetime.date(2017, 4, 28), datetime.date(2017, 1, 18), datetime.date(2016, 7, 15), datetime.date(2017, 11, 1), datetime.date(2017, 8, 9), datetime.date(2016, 10, 7)]
[datetime.date(2016, 8, 5), datetime.date(2016, 10, 8), datetime.date(2016, 10, 1), datetime.date(2017, 11, 3), datetime.date(2017, 5, 5), datetime.date(2017, 2, 13), datetime.date(2016, 7, 29), datetime.date(2017, 11, 18), datetime.date(2017, 8, 25), datetime.date(2016, 10, 24)]


In [18]:
search = pd.DataFrame(data={
    'date_created': search_date,
    'city_id': city_id,
    'promotion_id': promotion_id,
    'rooms': rooms,
    'check_in': check_in,
    'check_out': check_out,
    'id': search_id})

Take a look at the sample of the data.

In [24]:
search

,date_created,city_id,promotion_id,rooms,check_in,check_out,id
0,2016-06-10,28,0,4,2016-07-08,2016-08-05,157106
1,2016-09-23,103,0,1,2016-10-06,2016-10-08,374555
2,2016-08-04,66,0,1,2016-09-05,2016-10-01,973252
3,2017-10-04,45,0,3,2017-10-17,2017-11-03,265382
4,2017-04-12,63,1,2,2017-04-28,2017-05-05,687075
...,...,...,...,...,...,...,...
4995,2017-03-29,87,0,3,2017-04-06,2017-04-13,339084
4996,2017-01-19,93,0,3,2017-03-02,2017-03-10,869951
4997,2016-10-13,3,1,3,2016-10-27,2016-11-19,216852
4998,2016-10-17,95,0,1,2016-10-21,2016-10-26,253871


- Next, we will work on the reservation table.
- Use `np.random.choice` to select the actual reservations from the search table.
- Not all the reservations have been cancelled so you can use `np.random.binomial` to randomly pick the ones were cancelled and leave the others as `np.nan`. Use the `cancel_rate` as the probability for the binomial distribution.
- The same user might booked more than once so set `replace=True` when you sample the `user_id` from the range of all users

In [177]:
np.random.seed(0)

# Select random reservations from the search request table
# We want to keep the `check_in` column to further calculate the `date_cancelled`
reservation_index = np.random.choice(search.index.values, size = num_reservation)
reservation = search.loc[reservation_index, ['date_created', 'check_in', 'id']]

# Create a random `date_cancelled` between `date_created` and `check_in`
date_cancelled = []

for x in range(len(reservation_index)):
    date_cancelled.append(f.date_between_dates(reservation.iloc[x,0], reservation.iloc[x,1]))

# Update the date_cancelled column according to the binomial distribution
cancel_rate = 0.05

reservation['date_cancelled'] = date_cancelled

date_cancelled2 = []
for x in range(reservation.shape[0]):
    date_cancelled2.append(np.random.choice([reservation.iloc[x,-1], np.nan], 1, p = [0.95, 0.05]))

reservation['date_cancelled'] = [x[0] for x in date_cancelled2]
# reservation['date_cancelled'] = pd.to_datetime(reservation['date_cancelled'])

# Rename the id column from search dataframe as the search_request_id
reservation['search_request_id'] = search['id']

# reservation id column, select random numbers from a range
reservation['id'] = np.random.randint(1, 1e6, size = num_reservation)

# Drop the check_in column
reservation.drop(['check_in'], axis = 1, inplace = True)

# User id column, select random numbers from a range
# One user might have multiple bookings so use `replace=True`
# Your code here

reservation['user_id'] = np.random.choice(1000, size = num_reservation, replace = True)
reservation.head(40)

,date_created,id,date_cancelled,search_request_id,user_id
2732,2016-11-17,774161,2016-12-01,608478,103
2607,2016-12-31,865205,2017-01-04,547890,267
1653,2016-10-14,809179,2016-11-06,34851,312
3264,2016-11-04,995305,2016-11-06,361114,447
4931,2017-07-19,583889,2017-07-23,636542,142
4859,2017-05-07,975663,2017-06-10,420892,288
1033,2017-02-22,466145,2017-02-23,326631,193
4373,2016-07-19,690783,2016-07-24,840297,121
3468,2017-01-12,116565,2017-01-13,271585,373
705,2016-12-18,315160,2016-12-24,31306,833


In [ ]:
#not sure what is going on here
for x in reservation_index:
    print(reservation.loc[x,'date_created'])
    

Take a look at the sample of the data.

In [178]:
reservation.sample(10)

,date_created,id,date_cancelled,search_request_id,user_id
1672,2016-12-17,852153,2016-12-19,717261,476
3091,2017-07-08,227124,NaN,502908,755
770,2017-02-18,862517,2017-03-06,449879,593
3968,2016-10-13,92811,2016-10-22,841406,773
2965,2017-10-05,842099,2017-10-14,290197,781
2721,2017-01-26,597822,2017-01-26,608916,315
2711,2017-02-20,767951,2017-03-07,454897,219
2435,2016-12-17,98548,2017-01-02,123212,91
732,2016-06-30,24392,2016-07-31,934920,371
4370,2016-08-09,146542,NaN,724656,794


### Execute queries on pandas dataframe

**1.** List the top 20 cities by percentage share of total bookings between 2017-01-01 and 2017-08-30 (be sure to include each city’s share in your output)

**Note:** in sqlite, integer divided by another integer will be rounded to the closest integer. Times the integer by 1.0 to convert it to a float.

In [218]:
import pandasql as ps

q1 = """
    select city_id,
    count(*) * 1.0 / (select count(*) from reservation
    where reservation.date_created between '2017-01-02' and '2017-08-29') as city_share
    from reservation join search on reservation.search_request_id = search.id 
    where reservation.date_created between '2017-01-02' and '2017-08-29'
    group by city_id
    order by city_share desc
    limit 20;
     """

# globals will give you access to all the dataframes defined in the current notebook
ps.sqldf(q1, globals())

,city_id,city_share
0,108,0.024896
1,112,0.020747
2,111,0.020747
3,96,0.020747
4,92,0.020747
5,4,0.020747
6,110,0.016598
7,90,0.016598
8,87,0.016598
9,76,0.016598


**2.** For each of these top 20 cities, list their respective percentages of bookings that were cancelled between 2017-01-01 and 2017-08-30

**Note:** calculating the percentages of cancelled and non-cancelled reservations would involve multiple sub-queries so `CASE WHEN` query might be an easy solution here. Take the following query as an example:
```
SUM(
     CASE
         WHEN date_cancelled ISNULL 
         THEN 1 
         ELSE 0 
     END))
```
When the `date_cancelled` column is null, we count it as 1, otherwise we count it as 0. So eventually we will get the total number of cancellations w/o using sub-queries.

In [225]:
q2 = """  
     select city_id,  
     SUM(
     CASE
         WHEN reservation.date_cancelled ISNULL 
         THEN 1 
         ELSE 0 
     END) * 1.0 / count(*) as perc_cancelled
     from search join reservation on search.id = reservation.search_request_id
     where reservation.date_created between '2017-01-02' and '2017-08-29' 
     group by city_id
     order by perc_cancelled desc
     limit 20;
     """

print(ps.sqldf(q2, locals()))

    city_id  perc_cancelled
0        86        1.000000
1        79        1.000000
2        44        1.000000
3        17        0.500000
4        15        0.500000
5       101        0.333333
6        88        0.333333
7        52        0.250000
8        40        0.250000
9        27        0.250000
10       20        0.250000
11      119        0.000000
12      117        0.000000
13      116        0.000000
14      115        0.000000
15      114        0.000000
16      113        0.000000
17      112        0.000000
18      111        0.000000
19      110        0.000000


**3.** For each week between 2017-01-01 and 2017-08-30, list the percentage breakdown of bookings made by repeat customers vs. first-time customers w/ promotion vs. first-time customers w/o promotion

**Note:** To extract the date from timestamp, use the `strftime()` function in SQL. i.e `strftime('%W', date_created)`

In [234]:
q3 = """
    SELECT L.week, customer_group, COUNT(*) * 1.0 / WK.week_total AS percentage
    FROM
        (SELECT 
             strftime('%W', R.date_created) AS week,
             CASE 
                 WHEN R.date_created > R.first_time THEN 'repeated user'
                 WHEN R.date_created = R.first_time AND promotion_id IS NULL THEN 'first time user w/o promotion' 
                 ELSE 'first time user w/ promotion' 
             END AS customer_group
        FROM
            (SELECT * 
            FROM reservation JOIN
                (SELECT user_id, MIN(date_created) AS first_time 
                FROM reservation 
                GROUP BY user_id) AS F
                    ON reservation.user_id = F.user_id) AS R INNER JOIN 
                search
                    ON R.search_request_id = search.id
    WHERE R.date_created BETWEEN '2017-01-02' AND '2017-08-29') AS L INNER JOIN 
        (SELECT strftime('%W', date_created) AS week, count(*) as week_total 
        FROM reservation
        WHERE date_created BETWEEN '2017-01-02' AND '2017-08-29'
        GROUP BY strftime('%W', date_created)) AS WK
            ON L.week = WK.week
    GROUP BY L.week, customer_group;
     """

ps.sqldf(q3, locals())

,week,customer_group,percentage
0,01,first time user w/ promotion,0.900000
1,01,repeated user,0.100000
2,02,first time user w/ promotion,0.818182
3,02,repeated user,0.181818
4,03,first time user w/ promotion,0.600000
...,...,...,...
61,33,repeated user,0.833333
62,34,first time user w/ promotion,0.833333
63,34,repeated user,0.166667
64,35,first time user w/ promotion,0.500000


### Solutions

#### Data Preparation - Search Request

In [ ]:
# Fix random seed to make the result reproducible
np.random.seed(0)

num_users = 2000 # Number of users
num_search = 5000 # Number of searches
num_reservation = int(num_search * 0.1) # Assume 10% of the search got converted to reservation
num_cities = 120  # Number of cities
num_promotion = 20 # Number of different promotions
num_rooms = 5  # Largest reservation
most_days_in_advanced = 50 # customer can book at most 50 days after the search date to check in
most_days_stay = 30 # max stay to 8 days
promotion_rate = 0.1 # 10% of the people use promotion

# search_id
search_id = np.random.choice(range(1, int(1e6)), size = num_search, replace=False)

# city_id
city_id = np.random.randint(1, num_cities, size = num_search)

# number of rooms
rooms = np.random.randint(1, num_rooms, size = num_search)

# search_date
# Start date and end date of the fake data
start_date = dt.date(2016,6,6)
end_date = dt.date(2017,10,10)
# Use the `date_between_dates` method
f = Faker()
search_date = [f.date_between_dates(start_date, end_date) for i in range(num_search)]

# promotion_id
# use `np.random.binomial` to randomly pick the ones w/ promotion
tmp1 = np.random.randint(1, num_promotion, size = num_search)
tmp2 = np.random.binomial(1, promotion_rate, size=num_search)
promotion_id = tmp1 * tmp2
promotion_id = [x if x>0 else np.nan for x in promotion_id]

# check_in & check_out date
# use `dt.timedelta` function to create a time difference and add it to the original date
tmp1 = np.random.randint(1, most_days_in_advanced, size=num_search)
tmp2 = np.random.randint(1, most_days_stay, size=num_search)
check_in = list(map(lambda x, y: x+dt.timedelta(days=int(y)), search_date, tmp1))
check_out = list(map(lambda x, y: x+dt.timedelta(int(y)), check_in, tmp2))

#### Data Preparation - Reservations

In [ ]:
np.random.seed(0)

# Select random reservations from the search request table
# We want to keep the `check_in` column to further calculate the `date_cancelled`
reservation_index = np.random.choice(search.index.values, size = num_reservation)
reservation = search.loc[reservation_index, ['date_created', 'check_in', 'id']]

# Create a random `date_cancelled` between `date_created` and `check_in`
reservation['date_cancelled']= list(map(lambda x, y: f.date_between_dates(x, y),
      reservation['date_created'], reservation['check_in']))

# Update the date_cancelled column according to the binomial distribution
cancel_rate = 0.05
reservation['date_cancelled'] = reservation['date_cancelled']\
                                .apply(lambda x: np.nan if np.random.binomial(1, cancel_rate, size=1)[0]==0 else x)

# Rename the id column from search dataframe as the search_request_id
reservation = reservation.rename(columns={'id':'search_request_id'})

# reservation id column, select random numbers from a range
reservation_id = np.random.choice(range(1, int(1e5)), size = num_reservation, replace=False)
reservation['id'] = reservation_id

# Drop the check_in column
reservation.drop('check_in', inplace=True, axis=1)

# User id column, select random numbers from a range
# One user might have multiple bookings so use `replace=True`
reservation['user_id'] = np.random.choice(range(1, num_users), size = num_reservation, replace=True)

#### SQL Queries

In [ ]:
q1 = """
     SELECT 
         city_id, 
         COUNT(*) * 1.0 / (SELECT COUNT(*) 
                           FROM reservation 
                           WHERE date_created BETWEEN '2017-01-02' AND '2017-08-29') AS city_share
     FROM reservation JOIN search
         ON reservation.search_request_id = search.id
     WHERE reservation.date_created BETWEEN '2017-01-02' AND '2017-08-29'
     GROUP BY city_id
     ORDER BY city_share DESC
     LIMIT 20;
     """

In [ ]:
q2 = """     
     SELECT city_id, 
            COUNT(*) * 1.0 / 
                 (SELECT COUNT(*) 
                 FROM reservation 
                 WHERE 
                     date_created BETWEEN '2017-01-02' AND '2017-08-29') AS city_share,
             1 - SUM(
                     CASE 
                         WHEN date_cancelled ISNULL 
                             THEN 1 
                             ELSE 0 
                     END) * 1.0 / COUNT(*) AS cancel_rate
     FROM reservation INNER JOIN search ON reservation.search_request_id = search.id
     WHERE reservation.date_created BETWEEN '2017-01-02' AND '2017-08-29'
     GROUP BY city_id
     ORDER BY city_share DESC
     LIMIT 20;
     """

In [ ]:
q3 = """
    SELECT L.week, customer_group, COUNT(*) * 1.0 / WK.week_total AS percentage
    FROM
        (SELECT 
             strftime('%W', R.date_created) AS week,
             CASE 
                 WHEN R.date_created > R.first_time THEN 'repeated user'
                 WHEN R.date_created = R.first_time AND promotion_id IS NULL THEN 'first time user w/o promotion' 
                 ELSE 'first time user w/ promotion' 
             END AS customer_group
        FROM
            (SELECT * 
            FROM reservation JOIN
                (SELECT user_id, MIN(date_created) AS first_time 
                FROM reservation 
                GROUP BY user_id) AS F
                    ON reservation.user_id = F.user_id) AS R INNER JOIN 
                search
                    ON R.search_request_id = search.id
    WHERE R.date_created BETWEEN '2017-01-02' AND '2017-08-29') AS L INNER JOIN 
        (SELECT strftime('%W', date_created) AS week, count(*) as week_total 
        FROM reservation
        WHERE date_created BETWEEN '2017-01-02' AND '2017-08-29'
        GROUP BY strftime('%W', date_created)) AS WK
            ON L.week = WK.week
    GROUP BY L.week, customer_group;
     """